In [ ]:
# code
import param
import panel as pn
import ipywidgets as widgets
import hvplot.pandas

from scripting                          import *
from eventdetection.processor.alignment import MeasureEndAlignmentTask
from peakfinding.processor.projection   import PeakProjectorTask
from peakcalling.tohairpin              import PiecesPeakGridFit, Pivot, matchpeaks
from cleaning.processor                 import DataCleaningException

Tasks.cleaning.let(minextent = 0.05, maxsaturation = 100., maxpingpong = 10)

def beads(track, *tasks, distance = MeasureEndAlignmentTask.distance):
    "returns beads with some tasks applied"
    if track.tasks.subtraction is None:
        track.tasks.subtraction = track.cleaning.fixed()
    return track.apply(
        track.tasks.subtraction,
        Tasks.cleaning,
        MeasureEndAlignmentTask(distance = distance),
        *tasks,
        copy = True
    )

def peaks(track, cyclemincount = None):
    "returns peaks with some tasks applied"
    proj = PeakProjectorTask()
    if cyclemincount is None:
        cyclemincount = 1.5
    proj.projector.aggregate.cyclemincount = cyclemincount
    return beads(track, proj)

class TrackDisplayer:
    "display tracks"
    def __init__(self, tracks):
        self.tracks = tracks
        self.out    = widgets.Output(layout={'border': '1px solid black'})
        self.ccount = widgets.FloatSlider(
            value = 1.5,
            min   = 0.5,
            max   = 5.0,
            step  = 0.1,
            description='Min events (default = 1.5):',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        self.ccount.observe(self._onchangebead, names = "value")  
        self.trackw = widgets.Dropdown(
            options = [""],
            value   = "",
            description = "Track:",
            disabled    = False
        )
        self.trackw.on_displayed(self._ondisplayedtrack)
        self.trackw.observe(self._onchangetrack, names = "value")
        self.beadw = widgets.Dropdown(
            options= ["2"],
            value='2',
            description='Bead:',
            disabled= False,
        )
        self.beadw.observe(self._onchangebead, names = "value")
        self.good  = {}
        self.keys  = '', 0
        self.width = 500, 300

    def display(self):
        "display the graphs & the widgets"
        return widgets.VBox([widgets.HBox([self.trackw, self.beadw, self.ccount]), self.out])
        
    def draw(self, key, bead, ccount):
        "display peaks & cycles"
        if key not in self.tracks:
            return hv.Text(.5, .5, "Select a Track")
        track = self.tracks[key]
        dispbeads = lambda x: (
            beads(track, **({} if x else {'distance': None}))
            [...,...]
            .withphases(1,8)
            .display.display()[bead]
            .Curve.I.options(line_color = 'deepskyblue' if x else 'red')
        )
        return (
            (dispbeads(False) * dispbeads(True)).options(width = self.width[0])
            + (
                peaks(track, ccount)
                .display(zero = False)
                .display()[bead]
                .options(invert_axes = True, width= self.width[1])
            )
        )

    def _ondisplayedtrack(self, _):
        try:
            self.trackw.options = list(self.tracks)
        except:
            pass
        self.trackw.value   = self.trackw.options[0]
        self._onchangetrack({"new": self.trackw.options[0]})

    def _onchangetrack(self, change):
        key = change['new']
        if key not in self.good and self.tracks.get(key, None):
            track = self.tracks[key]
            if track.tasks.subtraction is None:
                track.tasks.subtraction = track.cleaning.fixed()
            self.good[key]  = [str(i) for i in track.cleaning.good()]
        self.beadw.options  = self.good.get(key, [])
        self.beadw.value    = "0" if len(self.beadw.options) == 0 else self.beadw.options[0]
        self._onchangebead(_)
        
    def _onchangebead(self, _):
        try:
            keys = self.trackw.value, int(self.beadw.value), self.ccount.value
        except:
            keys = None
        if keys != self.keys:
            self.keys = keys
            self.out.clear_output()
            with self.out:
                if keys is None:
                    display(widgets.Label("Please wait !"))
                elif len(self.beadw.options) == 0:
                    display(widgets.Label("No good beads in this track"))
                else:
                    display(self.draw(*keys))

class FittedTrackDisplayer(TrackDisplayer):
    "display fitted tracks"
    def __init__(self, tracks, **kwa):
        super().__init__(tracks)
        self.delta  = sum(abs(i) for i in kwa.pop("range", (0, 0)))
        self.fitter = self.instantiatefitter(**kwa)

    def draw(self, key, bead, ccount):
        "display peaks & data"
        pks = peaks(self.tracks[key], ccount)
        dtl = pks.detailed(bead)
        lst = pks[bead]['peaks']
        opt = self.fitter.optimize(lst)
        dtl.details.transform(opt[0][1:])
        out = hv.Overlay([
            dtl.display(zero = False).display(),
            *(
                hv.Scatter((i, 0.1)).options(size = 10)
                for i in self.fitter.reconstruct(opt)
            )
        ])

        theo  = np.concatenate([i.peaks for i in self.fitter.pieces])
        theo -= self.fitter.pieces[0].peaks[-1]
        rec   = np.concatenate(self.fitter.reconstruct(opt))
        rec  -= self.fitter.pieces[0].peaks[-1]
        ids   = [
            matchpeaks(i.peaks, (lst-j[2])*j[1], self.fitter.window)
            for i, j in zip(self.fitter.pieces, opt)
        ]
        exp   = [
            np.full(len(i.peaks), np.NaN, dtype = 'f4')
            for i in self.fitter.pieces
        ]
        rsize = np.diff(rec[len(exp[0])-1:][:-1])[0]-self.delta
        for j, k in zip(exp, ids):
            j[k[k>=0]] = lst[k>=0]
        frame = pd.DataFrame({
            'Experiment (µm)': np.concatenate(exp),
            'Theory (base)': theo,
            'Reconstructed (base)': rec
        })
        out   = (
            out.options(
                width = self.width[0],
                title_format = "Repeat region size: {:.0f} bases".format(rsize)
            )
            
            + hv.Table(frame.fillna("")).options(width = self.width[1])
        )
        return out

    @staticmethod
    def instantiatefitter(dna, oligos, div):
        "create the fitter instance"
        seq = next(sequences.read(dna))[1]
        ols = [i.strip() for i in open(oligos).readlines()[:-1]]
        pos = sequences.peaks(seq, ols)
        pos = np.array(list(pos[~pos['orientation']]['position'])+[len(seq)])
        fit = PiecesPeakGridFit(ordered = True)
        fit.appendpiece(
                peaks      = pos[:div],
                stretch    = (1e3, 200, 100),
                strandsize = pos[-1],
        )
        fit.appendpiece(
                peaks      = pos[div:],
                pivot      = Pivot.top,
                stretch    = (1e3, 200, 100),
                strandsize = pos[-1],
        )
        return fit

class FoVDisplay(param.Parameterized):
    "Display FoV results"
    fov              = param.ListSelector(default=[], objects = [], precedence= 0)
    event_count      = param.Number(doc = "event count", default = 1.5, bounds = (.5, 5.), precedence= 2)
    peak_count       = param.Number(doc = "peak count", default = 5, bounds = (3, 20), precedence= 3)
    identified_count = param.Number(doc = "identified peaks count", default = 2, bounds = (0, 10), precedence= 4)
    bin_size         = param.Number(doc = "bin size", default = 3, bounds = (3, 50), precedence= 1)
    def __init__(self, tracks = None, **kwa):
        self.tracks    = tracks if tracks else TRACKS
        FoVDisplay._FoVDisplay__params['fov'].objects = list(self.tracks)
        FoVDisplay._FoVDisplay__params['fov'].default = [next(iter(self.tracks))]
        
        if not kwa:
            kwa = dict(range = RANGE, dna = DNA, oligos = OLIGOS, div = DIV)
        name  = kwa.pop("name", "Field of View")
        self.__good    = {}
        self.__oldcnt  = None
        self.delta     = sum(abs(i) for i in kwa.pop("range", (0, 0)))
        self.__fitter  = FittedTrackDisplayer.instantiatefitter(**kwa)
        super().__init__(name = name)

    def compute(self, pks, bead):
        "display peaks & data"
        found= pks[bead]['peaks']
        opt  = self.__fitter.optimize(found)
        rec  = np.concatenate(self.__fitter.reconstruct(opt))
        rec -= self.__fitter.pieces[0].peaks[-1]
        exp  = [
            np.full(len(i.peaks), np.NaN, dtype = 'f4')
            for i in self.__fitter.pieces
        ]
        
        ids   = [
            matchpeaks(i.peaks, (found-j[2])*j[1], self.__fitter.window)
            for i, j in zip(self.__fitter.pieces, opt)
        ]
        return np.diff(rec[len(exp[0])-1:][:-1])[0]-self.delta, len(found), [(i>=0).sum() for i in ids]

    def display(self):
        return pn.Column(self.param, self.view)

    @param.depends('fov', 'event_count', "bin_size", "peak_count", "identified_count")
    def view(self):
        key = self.fov
        if not (set(key) & set(self.tracks)):
            key = [next(iter(self.tracks))]
        if not key:
            return pn.pane.Str(f"Choose an FoV")

        if self.event_count != self.__oldcnt:
            self.__good.clear()
            self.__oldcnt = self.event_count

        for k  in key:
            track = self.tracks[k]
            if track.tasks.subtraction is None:
                track.tasks.subtraction = track.cleaning.fixed()
            if k not in self.__good:
                self.__good[k] = {i: None for i in track.cleaning.good()}
                pks  = peaks(track, self.event_count)
                for i in list(self.__good[k]):
                    self.__good[k][i] = self.compute(pks, i)
        sel = {
            (k, j): i
            for k in key for j, i in self.__good[k].items()
            if (
                i is not None
                and i[1] >= self.peak_count
                and all(j >= self.identified_count for j in i[2])
            )
        } 

        good = np.array([i[0] for i in sel.values()])
        if len(good) == 0:
            return pn.pane.Str(f"Empty FoVs")
        table = pd.DataFrame({
            'FoV': [k for k in key for i in self.__good[k]],
            'bead': [i for k in key for i in self.__good[k]],
            'selected': [(k,i) in sel for k in key for i in self.__good[k]],
            'region': [i[0] for k in key for i in self.__good[k].values()],
            'peaks': [i[1] for k in key for i in self.__good[k].values()],
            'found (bottom)': [i[2][0] for k in key for i in self.__good[k].values()],
            'found (top)': [i[2][1] for k in key for i in self.__good[k].values()]
        })
        rng  = int(good.min())-2, int(good.max())+2
        bins = np.linspace(rng[0], rng[1], max((rng[1]-rng[0])//self.bin_size+1, 2))
        return pn.panel((
            hv.Histogram(np.histogram(good, bins = bins))
            .redim(x = "Repeat Region Size")
            .options(width= 500)
            +table.hvplot.table(list(table.columns))
        ))

In [ ]:
DNA    = "/media/samba-shared/data/tmp/pol/Repeatnumber/plasmid.fa"       # sequence in fasta format
OLIGOS = "/media/samba-shared/data/tmp/pol/Repeatnumber/OligoList.txt" # list of oligos, 1 per line
DIV    = 7       # the number of peaks below the repeat region
RANGE  = 1, -23 # distance to the repeat region from the last bottom peak and the first top peak

# The tracks:
# * 1st argument is the path to the tracks. Don't forget the '*.trk' at the end
# * 2nd argument is optional. It uses regular expressions to extract a simpler name from track file names.
TRACKS = TracksDict("/media/samba-shared/data/tmp/pol/Repeatnumber/20190207/*.trk", match = ".*(FOV\d+).*")
#TRACKS['plasmid'] = Track(path = "/data/tracks/repeats/plasmid_FmrI_2018-09-10.trk")

In [ ]:
# Display RAW data
TrackDisplayer(TRACKS).display()

In [ ]:
%%opts Curve[tools=['hover']]
FittedTrackDisplayer(TRACKS, range = RANGE, dna = DNA, oligos = OLIGOS, div = DIV).display()

In [ ]:
# Display all values for selected FoVs
FoVDisplay().display()